In [70]:
# Import needed libraries 

import pandas as pd 
import os
from datetime import timedelta
import numpy as np 
from scipy.signal import argrelextrema
import alpaca_trade_api as tradeapi 
import matplotlib.pyplot as plt 
import matplotlib.dates as mpdates
from mplfinance.original_flavor import candlestick_ohlc
from dotenv import load_dotenv
from itertools import islice

In [71]:
# Load .env file

load_dotenv()

True

In [72]:
# Set Alpaca API key and secret passwords

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [73]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [74]:
# Parameters for Stock Data from Alpacas
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what stock symbol is trading

stock_symbol = "QQQ"

# Identify what start date to begin data analysis
# JAN 27 27 2021 GameStop event (GME) are we still doing this?

start_date = pd.Timestamp("2021-02-03", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2022-02-03", tz="America/New_York").isoformat()

In [75]:
# Call stock data from identified period of time

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 46815 entries, 2021-02-03 09:00:00+00:00 to 2022-02-03 00:55:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        46815 non-null  float64
 1   high         46815 non-null  float64
 2   low          46815 non-null  float64
 3   trade_count  46815 non-null  int64  
 4   open         46815 non-null  float64
 5   volume       46815 non-null  int64  
 6   vwap         46815 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 2.9 MB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-02-03 09:00:00+00:00,330.24,330.33,330.11,39,330.11,3383,330.227245
2021-02-03 09:05:00+00:00,329.92,330.22,329.88,38,330.22,5358,330.052469
2021-02-03 09:10:00+00:00,329.99,330.07,329.95,15,329.97,2615,329.999281
2021-02-03 09:15:00+00:00,329.91,329.91,329.75,20,329.89,1424,329.869747
2021-02-03 09:20:00+00:00,329.83,329.89,329.71,28,329.87,2871,329.788189


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2022-02-03 00:35:00+00:00,360.5000,360.77,360.44,105,360.53,23590,360.548267
2022-02-03 00:40:00+00:00,360.7400,360.74,360.56,85,360.64,18651,360.625474
2022-02-03 00:45:00+00:00,360.5300,360.75,360.52,83,360.70,16244,360.650788
2022-02-03 00:50:00+00:00,360.6400,360.66,360.53,127,360.54,19034,360.605244
2022-02-03 00:55:00+00:00,360.1701,360.59,360.15,209,360.56,43861,360.314995


In [76]:
# Function to prepare daily stock data to identify double top/bottom patterns and prepare for targets
# Includes establishing a polynomial fit and assigning new columns for localized min/max

# Polynomial Degree

polynomial_degree = 75

# Updated DF with appropriate columns (polfits, mins, max, targets)

updated_stock_data = stock_data

updated_stock_data["poly_fit_open"] = 0
updated_stock_data["poly_fit_high"] = 0
updated_stock_data["poly_fit_low"] = 0
updated_stock_data["poly_fit_close"] = 0
updated_stock_data["min"] = 0
updated_stock_data["max"] = 0
updated_stock_data["double_top"] = 0
updated_stock_data["double_bot"] = 0

# Extract ehd ate from the timestamp

stock_data.reset_index(inplace=True)

stock_data["timestamp"] = pd.to_datetime(stock_data["timestamp"])

stock_data["date"] = stock_data["timestamp"].dt.date

grouped = stock_data.groupby("date")

display(stock_data.head())
display(grouped.head())


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,min,max,double_top,double_bot,date
0,2021-02-03 09:00:00+00:00,330.24,330.33,330.11,39,330.11,3383,330.227245,0,0,0,0,0,0,0,0,2021-02-03
1,2021-02-03 09:05:00+00:00,329.92,330.22,329.88,38,330.22,5358,330.052469,0,0,0,0,0,0,0,0,2021-02-03
2,2021-02-03 09:10:00+00:00,329.99,330.07,329.95,15,329.97,2615,329.999281,0,0,0,0,0,0,0,0,2021-02-03
3,2021-02-03 09:15:00+00:00,329.91,329.91,329.75,20,329.89,1424,329.869747,0,0,0,0,0,0,0,0,2021-02-03
4,2021-02-03 09:20:00+00:00,329.83,329.89,329.71,28,329.87,2871,329.788189,0,0,0,0,0,0,0,0,2021-02-03


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,min,max,double_top,double_bot,date
0,2021-02-03 09:00:00+00:00,330.24,330.33,330.11,39,330.11,3383,330.227245,0,0,0,0,0,0,0,0,2021-02-03
1,2021-02-03 09:05:00+00:00,329.92,330.22,329.88,38,330.22,5358,330.052469,0,0,0,0,0,0,0,0,2021-02-03
2,2021-02-03 09:10:00+00:00,329.99,330.07,329.95,15,329.97,2615,329.999281,0,0,0,0,0,0,0,0,2021-02-03
3,2021-02-03 09:15:00+00:00,329.91,329.91,329.75,20,329.89,1424,329.869747,0,0,0,0,0,0,0,0,2021-02-03
4,2021-02-03 09:20:00+00:00,329.83,329.89,329.71,28,329.87,2871,329.788189,0,0,0,0,0,0,0,0,2021-02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46803,2022-02-03 00:00:00+00:00,360.44,360.65,360.05,238,360.14,88125,360.233396,0,0,0,0,0,0,0,0,2022-02-03
46804,2022-02-03 00:05:00+00:00,360.14,360.49,360.02,92,360.49,10385,360.195700,0,0,0,0,0,0,0,0,2022-02-03
46805,2022-02-03 00:10:00+00:00,360.50,360.56,360.18,189,360.18,59068,360.406148,0,0,0,0,0,0,0,0,2022-02-03
46806,2022-02-03 00:15:00+00:00,360.66,360.85,360.47,197,360.47,40473,360.706998,0,0,0,0,0,0,0,0,2022-02-03


In [77]:
# # Read through code to identify double top/bottom and assign to target columns.

for date, group in grouped:

    # Extract data for the polynomial fit
    
    x_data = np.arange(len(group))
    y_data_open = group["open"].values
    y_data_high = group["high"].values
    y_data_low = group["low"].values
    y_data_close = group["close"].values

    # Apply polyfit function

    polynomial_coefficients_open = np.polyfit(x_data, y_data_open, polynomial_degree)
    polynomial_coefficients_high = np.polyfit(x_data, y_data_high, polynomial_degree)
    polynomial_coefficients_low = np.polyfit(x_data, y_data_low, polynomial_degree)
    polynomial_coefficients_close = np.polyfit(x_data, y_data_close, polynomial_degree)

    # Evaluate the polynomial for plotting

    y_polynomial_open = np.polyval(polynomial_coefficients_open, x_data)
    y_polynomial_high = np.polyval(polynomial_coefficients_high, x_data)
    y_polynomial_low = np.polyval(polynomial_coefficients_low, x_data)
    y_polynomial_close = np.polyval(polynomial_coefficients_close, x_data)

    # Identify local extrema for polynomial fit data (min and max)

    local_poly_minima = argrelextrema(y_polynomial_close, np.less, order=5)[0]
    local_poly_maxima = argrelextrema(y_polynomial_close, np.greater, order=5)[0]

    # Mark rows with local min as -1 and max as 1

    # Write the data to stock data DF

    updated_stock_data.loc[group.index, "poly_fit_open"] = y_polynomial_open
    updated_stock_data.loc[group.index, "poly_fit_high"] = y_polynomial_high
    updated_stock_data.loc[group.index, "poly_fit_low"] = y_polynomial_low
    updated_stock_data.loc[group.index, "poly_fit_close"] = y_polynomial_close
    updated_stock_data.loc[local_poly_minima, "min"] = -1
    # updated_stock_data.loc[group.index, "max"] = local_poly_maxima

    # Reset the index of the original updated ticer dataframe to conact with polynomial DF that does not include timeseries
    # This will ensure that the indexed integers of the poynomial fit align with the time each data point corresponds to

    # updated_stock_data = stock_data.reset_index()
    # updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')
    # updated_stock_data.head()

    # Add min and max column to the DF


/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/lib/polynomial.py:667: RuntimeWarning: overflow encountered in multiply
  scale = NX.sqrt((lhs*lhs).sum(axis=0))
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_4874/1358010429.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, y_data_open, polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_4874/1358010429.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, y_data_high, polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_4874/1358010429.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.pol

In [78]:
display(updated_stock_data.head(50))
# print(grouped)
# print(group)
# print(len(group))
# len(grouped)

group_sizes = stock_data.groupby(stock_data["timestamp"].dt.date).size()
print(group_sizes)

,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,min,max,double_top,double_bot,date
0,2021-02-03 09:00:00+00:00,330.240,330.33,330.11,39,330.1100,3383,330.227245,330.151767,330.337719,330.117214,330.195194,0,0,0,0,2021-02-03
1,2021-02-03 09:05:00+00:00,329.920,330.22,329.88,38,330.2200,5358,330.052469,330.099406,330.188430,329.918745,330.040855,-1,0,0,0,2021-02-03
2,2021-02-03 09:10:00+00:00,329.990,330.07,329.95,15,329.9700,2615,329.999281,330.032864,330.074616,329.834836,329.942733,-1,0,0,0,2021-02-03
3,2021-02-03 09:15:00+00:00,329.910,329.91,329.75,20,329.8900,1424,329.869747,329.960539,329.984965,329.789343,329.871657,-1,0,0,0,2021-02-03
4,2021-02-03 09:20:00+00:00,329.830,329.89,329.71,28,329.8700,2871,329.788189,329.886967,329.910336,329.747584,329.811927,-1,0,0,0,2021-02-03
5,2021-02-03 09:25:00+00:00,329.860,329.94,329.81,21,329.9300,3150,329.897730,329.814955,329.844714,329.698734,329.756718,-1,0,0,0,2021-02-03
6,2021-02-03 09:30:00+00:00,329.600,329.95,329.48,40,329.8400,5399,329.733469,329.746716,329.785176,329.644173,329.704633,-1,0,0,0,2021-02-03
7,2021-02-03 09:35:00+00:00,329.580,329.58,329.51,11,329.5100,507,329.542899,329.684357,329.731317,329.590153,329.657208,-1,0,0,0,2021-02-03
8,2021-02-03 09:40:00+00:00,329.590,329.59,329.59,2,329.5900,101,329.589901,329.629983,329.684439,329.543536,329.617161,-1,0,0,0,2021-02-03
9,2021-02-03 09:45:00+00:00,329.530,329.59,329.53,7,329.5900,1340,329.585634,329.585583,329.646712,329.509625,329.587258,-1,0,0,0,2021-02-03


timestamp
2021-02-03    180
2021-02-04    191
2021-02-05    179
2021-02-08    179
2021-02-09    188
             ... 
2022-01-28    180
2022-01-31    180
2022-02-01    192
2022-02-02    192
2022-02-03     12
Length: 256, dtype: int64


In [79]:
# # Reset the index of the original updated ticker dataframe to concat with the polynomial dataframe that does not include a timeseries
# # This will ensure that the indexed intergers of the ploynomial fit align with the time each data point corresponds to

# updated_stock_data = stock_data.reset_index()
# updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')
# updated_stock_data.head()


In [80]:
# # Add minima and maxima column to the DataFrame

# #updated_stock_data[columns = {'minima',"maxima"}] = 0  # Initialize 'minima' column with zeros
# updated_stock_data["minima"] = 0
# updated_stock_data["maxima"] = 0
# updated_stock_data.head()

# # Mark rows with local minima as 1 in the 'minima' column
# # Can we clean this up??? use one for loop with if statements?

# for index in local_poly_minima:
#     updated_stock_data.at[index, 'minima'] = -1

# for index in local_poly_maxima:
#     updated_stock_data.at[index, "maxima"] = 1

# #updated_stock_data.head(50)

In [81]:
# # Create Target Columns - Double Top Target & Double Bottom Target
# updated_stock_data["dbl_top_target"] = 0
# updated_stock_data["dbl_bot_target"] = 0

# updated_stock_data.head()
# updated_stock_data.tail()

In [82]:
# Function to prepare daily stock data to identify double top/bottom patterns and prepare for targets
# Includes establishing a polynomial fit and assigning new columns for localized min/max

# Polynomial Degree

polynomial_degree = 75

def polynomial_min_max_fit(stock_data, polynomial_degree):
    
    min_length = min(len(stock_data.index), len(stock_data['close']))
    x_data = np.arange(min_length)

    # Polynomial fitting
    polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
    polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
    polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
    polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)

    # Evaluate the polynomial for plotting
    y_polynomial_open = np.polyval(polynomial_coefficients_open, x_data)
    y_polynomial_high = np.polyval(polynomial_coefficients_high, x_data)
    y_polynomial_low = np.polyval(polynomial_coefficients_low, x_data)
    y_polynomial_close = np.polyval(polynomial_coefficients_close, x_data)

    # Identify local extrema for polynomial fit data (minima and maxima)
    local_poly_minima = argrelextrema(y_polynomial_close, np.less, order=5)[0]
    local_poly_maxima = argrelextrema(y_polynomial_close, np.greater, order=5)[0]

    # Convert the close price polynomial fit data into a dataframe
    # This is done for OHLC poly fit data

    poly_df = pd.DataFrame(y_polynomial_open)
    columns = ['poly_fit_open']
    poly_df.columns = columns
    poly_df = poly_df.assign(poly_fit_high = y_polynomial_high)
    poly_df = poly_df.assign(poly_fit_low = y_polynomial_low)
    poly_df = poly_df.assign(poly_fit_close = y_polynomial_close)

    # Reset the index of the original updated ticker dataframe to concat with the polynomial dataframe that does not include a timeseries
    # This will ensure that the indexed intergers of the ploynomial fit align with the time each data point corresponds to

    updated_stock_data = stock_data.reset_index()
    updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')
    updated_stock_data.head()

    # Add minima and maxima column to the DataFrame

    updated_stock_data["minima"] = 0
    updated_stock_data["maxima"] = 0
    updated_stock_data.head()

    # Mark rows with local minima as 1 in the 'minima' column

    for index in local_poly_minima:
        updated_stock_data.at[index, 'minima'] = -1

    for index in local_poly_maxima:
        updated_stock_data.at[index, "maxima"] = 1

    # Create Target Columns - Double Top Target & Double Bottom Target
        
    updated_stock_data["dbl_top_target"] = 0
    updated_stock_data["dbl_bot_target"] = 0

    return updated_stock_data, min_length, y_polynomial_close, local_poly_minima, local_poly_maxima

updated_stock_data, min_length, y_polynomial_close, local_poly_minima, local_poly_maxima = polynomial_min_max_fit(stock_data,polynomial_degree)

 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value


/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/lib/twodim_base.py:620: RuntimeWarning: overflow encountered in accumulate
  multiply.accumulate(tmp[:, 1:], out=tmp[:, 1:], axis=1)
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/lib/polynomial.py:667: RuntimeWarning: overflow encountered in multiply
  scale = NX.sqrt((lhs*lhs).sum(axis=0))
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/numpy/lib/polynomial.py:668: RuntimeWarning: invalid value encountered in divide
  lhs /= scale


LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
# Plot data
# Plot the stock data and identified minima

plt.figure(figsize=(15, 5))
plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

# # Plot polynomial fit

plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

# # Plot red dots at local minima and blue dots at local maxima

plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")

In [ ]:
# # Define the time independent DF for double top/bottom identification
# # Timestamp remains a column, but can identify pattern from peak to peak immediately

# time_independent_df = []
# time_independent_df = pd.DataFrame(time_independent_df, columns = ["timestamp","close","high","low","trade_count","open","volume","vwap","poly_fit_open","poly_fit_high","poly_fit_low","poly_fit_close","minima","maxima","dbl_top_target","dbl_bot_target"])

# for index, row in islice(updated_stock_data.iterrows(), 0, None):

#     # Assign to rows only those that contain local min or max

#     if (updated_stock_data.at[index,"minima"] == -1):
#         time_independent_df.loc[index] = row
#         a = updated_stock_data.iloc[index]["poly_fit_close"]
#     elif (updated_stock_data.at[index,"maxima"] == 1):
#         time_independent_df.loc[index] = row
#         a = updated_stock_data.iloc[index]["poly_fit_close"]

# # Reset time dependent index
# # Set new time independent index

# time_independent_df.reset_index(inplace = True)
# time_independent_df.rename(columns={"index":"time_dependent_index"}, inplace = True)

# # Data review
        
# display(time_independent_df.head(20))
# print()
# print(row)
# print()
# print(a)
# print(index)
# print()
# print(len(time_independent_df))

In [ ]:
# Define the time independent DF for double top/bottom identification
# Timestamp remains a column, but can identify pattern from peak to peak immediately

def time_independent_df(updated_stock_data):

    time_independent_df = []
    time_independent_df = pd.DataFrame(time_independent_df, columns = ["timestamp","close","high","low","trade_count","open","volume","vwap","poly_fit_open","poly_fit_high","poly_fit_low","poly_fit_close","minima","maxima","dbl_top_target","dbl_bot_target"])

    for index, row in islice(updated_stock_data.iterrows(), 0, None):

        # Assign to rows only those that contain local min or max

        if (updated_stock_data.at[index,"minima"] == -1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]
        elif (updated_stock_data.at[index,"maxima"] == 1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]

    # Reset time dependent index
    # Set new time independent index

    time_independent_df.reset_index(inplace = True)
    time_independent_df.rename(columns={"index":"time_dependent_index"}, inplace = True)

    return time_independent_df

time_independent_df = time_independent_df(updated_stock_data)

In [ ]:
# # Initialize variables to identify double top/bottom patterns
# # Time dependent variable x_0 will always begin at a local min/max which also coincides
# # With the start of a trend into a potential double top/bottom
    
# x_0 = 0

# # Initiation of double top/bottom variable
    
# a = 0

# # First peak/valley of double top/bottom pattern
    
# b = 0

# # Trough/peak of double top/bottom pattern

# c = 0

# # Second peak/vallye of double top/bottom pattern
    
# d = 0

# # Trigger of double top/bottom pattern
    
# e = 0

# # Final time increment to finalize and trigger double top signal
    
# x_f_min = 0
# x_f_max = 0

In [ ]:
# # Read through code to identify double top/bottom and assign to target columns.

# for index, row in islice(time_independent_df.iterrows(), 0, len(time_independent_df) - 4):

#     # Check for double top
#     # If found then assigns x_f final time for writing to targets in time dependent dataframe

#     if (time_independent_df.at[index,"minima"] == -1):
#         a = time_independent_df.iloc[index]["poly_fit_close"]
#         b = time_independent_df.iloc[index + 1]["poly_fit_close"]
#         c = time_independent_df.iloc[index + 2]["poly_fit_close"]
#         d = time_independent_df.iloc[index + 3]["poly_fit_close"]
#         e = time_independent_df.iloc[index + 4]["poly_fit_close"]
#         x_0 = time_independent_df.iloc[index]["time_dependent_index"]
#         print(a)
#         print(b)
#         print(c)
#         print(d)
#         print(e)
#         print(x_0)
#         if (time_independent_df.iloc[index + 1]["poly_fit_low"]) < d < (time_independent_df.iloc[index + 1]["poly_fit_high"]) and (e < c):
#             x_f = time_independent_df.iloc[index + 4]["time_dependent_index"]
#             while (x_0 < x_f + 1):
#                 updated_stock_data.at[x_0, "dbl_top_target"] = 1
#                 x_0 = x_0 + 1
#             break

#     # Check for double bottom
#     # If found then assigns x_f final time for writing to targets in time dependent dataframe
            
#     elif (time_independent_df.at[index,"maxima"] == 1):
#         a = time_independent_df.iloc[index]["poly_fit_close"]
#         b = time_independent_df.iloc[index + 1]["poly_fit_close"]
#         c = time_independent_df.iloc[index + 2]["poly_fit_close"]
#         d = time_independent_df.iloc[index + 3]["poly_fit_close"]
#         e = time_independent_df.iloc[index + 4]["poly_fit_close"]
#         x_0 = time_independent_df.iloc[index]["time_dependent_index"] 
#         print(a)
#         print(b)
#         print(c)
#         print(d)
#         print(e)
#         print(x_0)
#         if (time_independent_df.iloc[index+1]["poly_fit_low"] < d < time_independent_df.iloc[index+1]["poly_fit_high"]) and (e > c):
#             x_f_min = time_independent_df.iloc[index + 4]["time_independent_df"]
#             while (x_0 < x_f + 1):
#                 updated_stock_data.at[x_0, "dbl_bot_target"] = 1
#                 x_0 = x_0 + 1
#             break

# print(x_f)



In [ ]:
# Function to identify daily double top/bottom patterns
# Inherently there should only be 1 identification of the pattern as it is considered a trend reversal pattern
# The loop will break once the pattern is identified

def identify_double_patterns(time_independent_df, updated_stock_data):

    # Initialize variables to identify double top/bottom patterns
    # Time dependent variable x_0 will always begin at a local min/max which also coincides
    # With the start of a trend into a potential double top/bottom
    
    x_0 = 0

    # Initiation of double top/bottom variable
    
    a = 0

    # First peak/valley of double top/bottom pattern
    
    b = 0

    # Trough/peak of double top/bottom pattern

    c = 0

    # Second peak/vallye of double top/bottom pattern
    
    d = 0

    # Trigger of double top/bottom pattern
    
    e = 0

    # Final time increment to finalize and trigger double top signal
    
    x_f = 0

    # Read through code to identify double top/bottom and assign to target columns.

    for index, row in islice(time_independent_df.iterrows(), 0, len(time_independent_df) - 4):

        # Check for double top
        # If found then assigns x_f final time for writing to targets in time dependent dataframe

        if (time_independent_df.at[index,"minima"] == -1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"]
            print(a)
            print(b)
            print(c)
            print(d)
            print(e)
            print(x_0)
            if (time_independent_df.iloc[index + 1]["poly_fit_low"]) < d < (time_independent_df.iloc[index + 1]["poly_fit_high"]) and (e < c):
                x_f = time_independent_df.iloc[index + 4]["time_dependent_index"]
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_top_target"] = 1
                    x_0 = x_0 + 1
                break

        # Check for double bottom
        # If found then assigns x_f final time for writing to targets in time dependent dataframe
            
        elif (time_independent_df.at[index,"maxima"] == 1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"] 
            print(a)
            print(b)
            print(c)
            print(d)
            print(e)
            print(x_0)
            if (time_independent_df.iloc[index+1]["poly_fit_low"] < d < time_independent_df.iloc[index+1]["poly_fit_high"]) and (e > c):
                x_f_min = time_independent_df.iloc[index + 4]["time_independent_df"]
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_bot_target"] = 1
                    x_0 = x_0 + 1
                break
            
    return updated_stock_data, x_f

updated_stock_data, x_f = identify_double_patterns(time_independent_df,updated_stock_data)

print(x_f)



In [ ]:
# Data Checking

display(updated_stock_data.head(20))
print(updated_stock_data.loc[x_f])